### References
- Finetune Llama 3 for Sentiment Analysis (https://www.kaggle.com/code/lucamassaron/fine-tune-llama-3-for-sentiment-analysis)
- Finetune Llama 2 for Sentiment Analysis (https://www.kaggle.com/code/lucamassaron/fine-tune-llama-2-for-sentiment-analysis)

In [1]:
!nvidia-smi

Wed Oct 23 12:21:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Libraries

In [2]:
# %pip install -q -U torch --index-url https://download.pytorch.org/whl/cu117
%pip install -q -U -i https://pypi.org/simple/ bitsandbytes
%pip install -q -U transformers=="4.40.0"
%pip install -q -U accelerate
%pip install -q -U datasets
%pip install -q -U trl
%pip install -q -U peft
# %pip install -q -U tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.6 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 103.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 12.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import torch
from datasets import load_dataset

# Config

In [4]:
lang = 'eng'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') # Might not work on Kaggle
model_name = 

Disabling two features in PyTorch related to memory efficiency and speed during operations on the Graphics Processing Unit (GPU) specifically for the scaled dot product attention (SDPA) function.

In [5]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

# Data

## Load Data

In [8]:
print("Hugging Face token (https://huggingface.co/settings/tokens):")
hf_token = input()
!huggingface-cli login --token $hf_token

Hugging Face token (https://huggingface.co/settings/tokens):
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [10]:
data_files = {
    'train': f'preprocessed_data/train/{lang}.csv', 
    'val': f'preprocessed_data/val/{lang}.csv',
    'test': f'preprocessed_data/test/{lang}.csv',
}
dataset = load_dataset('alxxtexxr/SemEval2025-Task11-Dataset', data_files=data_files)

train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])
val_df = pd.DataFrame(dataset['val'])

print("DF columns:", list(train_df.columns))
print()
print("Train DF size:", len(train_df))
print("Validation DF size:", len(val_df))
print("Testing DF size:", len(test_df))

DF columns: ['Unnamed: 0', 'text', 'emotion', 'anger', 'fear', 'joy', 'sad', 'surprise']

Train DF size: 2214
Validation DF size: 554
Testing DF size: 116


## Create Prompt Data

In [81]:
prompt_template = """Analyze the emotion of the text within square brackets. Identify whether the emotion is one or a combination of the following: anger, fear, joy, sad, surprise, or neutral. 

[{text}] = {emotion}"""

def create_prompt(row):
    return prompt_template.format(text=row['text'], emotion=row['emotion']).strip()

def create_test_prompt(row):
    return prompt_template.format(text=row['text'], emotion="").strip()

train_df['prompt'] = train_df.apply(create_prompt, axis=1)
val_df['prompt'] = val_df.apply(create_test_prompt, axis=1)
test_df['prompt'] = test_df.apply(create_test_prompt, axis=1)

print("Train prompts:\n")
for prompt in train_df['prompt'].head(3):
    print(prompt)
    print("================================================================================================================================================================================================")
print()
print("Testing prompts:\n")
for prompt in test_df['prompt'].head(3):
    print(prompt)
    print("================================================================================================================================================================================================")

Train prompts:

Analyze the emotion of the text within square brackets. Identify whether the emotion is one or a combination of the following: anger, fear, joy, sad, surprise, or neutral. 

[I now have 12 of those canker sore suckers in my mouth along with a fever since friday.] = fear, sad
Analyze the emotion of the text within square brackets. Identify whether the emotion is one or a combination of the following: anger, fear, joy, sad, surprise, or neutral. 

[It just... went away.] = fear, sad, surprise
Analyze the emotion of the text within square brackets. Identify whether the emotion is one or a combination of the following: anger, fear, joy, sad, surprise, or neutral. 

[I naively walked up and stuck my head in the driver's window hole.] = fear, surprise

Testing prompts:

Analyze the emotion of the text within square brackets. Identify whether the emotion is one or a combination of the following: anger, fear, joy, sad, surprise, or neutral. 

[My mouth fell open `` No, no, no..

# Model